<center>
    <h1>Thrifting Alpha</h1>

    <h2>Revitalizing Tired Alpha Factors Using Ensemble Learning</h2>
    <img src="./images/logo-q-press-logo-red.png" alt="Quantopian" style="width: 200px;"/>
    
    <h4>Max Margenot</h4>
    <h4>Lead - Data Science/Academia</h4>
</center>

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company.  In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*

## What is an Alpha Factor?

* Also known as a Risk Factor
* Alternative risk premium

## Popularity of Factor Models

* "Smart Beta"
* Diversify with specific premia
* Analyze risk
* Increase returns

* 87% of institutional investors incorporating factors into investment process\*
* 67% of into risk management process\*
* 53% of into investment strategies\*


\*"The rise of factor investing." BlackRock. 7 Apr. 2017

## Constructing a Factor Model

* CAPM

$$ r_p = \alpha + \beta_{m} r_{m} $$

* Fama-French Factors

$$ r_p = \alpha + \beta_{m} r_{m} + \beta_{hml} r_{hml} + \beta_{smb} r_{smb} $$

* Alternative Factors

$$ r_p = \alpha + \beta_0 r_0 + \cdots + \beta_n r_n $$

## Using a Factor Model

* A few different ways to do this, once you have decided on your factors
* Compute exposures to your factor
* Use the exposures to create your portfolio

## Main Factors

* Value
* Momentum
* Quality
* Volatility
* Growth

## The Method

* Long-short Equity
    * Rank all securities in your universe from lowest factor exposure to highest
    * Go long on the top percentile
    * Go short on the bottom percentile

## The Universe

* Sufficient liquidity
    * No "hard-to-trade" securities

In [ ]:
universe = QTradableStocksUS()

* https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus

## The Point

* Predicting prices is hard
* Trying to capture the relative value of the long and short baskets
* Assuming that expected return of each security in the universe in proportional to factor value

## Example Factor: Momentum

In [ ]:
class Advanced_Momentum(CustomFactor):
        """ Momentum factor """
        inputs = [USEquityPricing.close,
                  Returns(window_length=126)]
        window_length = 252

        def compute(self, today, assets, out, prices, returns):
            out[:] = ((prices[-21] - prices[-252])/prices[-252] -
                      (prices[-1] - prices[-21])/prices[-21]) / np.nanstd(returns, axis=0)

## Example Factor: Momentum

<img src="./images/momentum_factor.png" alt="Momentum Factor" style="width: 800px;"/>

## Example Factor: Momentum

<img src="./images/momentum_factor_curve.png" alt="Momentum Factor" style="width: 800px;"/>

## Example Factor: Momentum

<img src="./images/momentum_factor_fingers.png" alt="Momentum Factor" style="width: 800px;"/>

## Other Factors in the Model

<img src="./images/many_curves.png" alt="Other Factors" style="width: 1500px;"/>

## Workflow
<img src="./images/quant_workflow.png" alt="Quant Equity Workflow" style="width: 800px;"/>

## Factor Aggregation

* Factors can be seen as individual constructed assets
* We can combine many to create a mega-factor
* Weighting scheme between factors is up to us

## Dynamic vs. Static Aggregation Methods

* Static: Predetermined aggregate factor weighting
* Dynamic: Changing weights based on their own performance

## Static Equal-Weighted Factor Portfolio

* Take all factors
* Standardize them
* Add them together

## Static Equal-Weighted Factor Portfolio

<img src="./images/equal_weighted_factor_portfolio.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Static Equal-Weighted Factor Portfolio

<img src="./images/equal_weighted_factor_portfolio_curve.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Static Equal-Weighted Factor Portfolio

<img src="./images/equal_weighted_factor_portfolio_fingers.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Ensemble Learning

* Obtain better predictive performance by aggregating many learning algorithms
* Very difficult to find a good trading algorithm

* Combining "meh" stuff into "okay" stuff

## Alpha Factors as Classifiers

* We can reinterpret an alpha factor as a classifier
* Relative value, classified as going up or going down based on predictor
    * High factor values indicate that we expect the value to go up
    * Low factor values indicate that we expect the value to go down
* Truncate these into a classification problem.

## AdaBoost (Adaptive Boosting)

$$ H(x) = sign\left(\sum_{t=1}^T \alpha_t h_t(x)\right) $$

* where each $h_t(x)$ is the output of a weak classifier and $\alpha_t$ is its weight
* Linear combination of classifiers inside a non-linear function

## Implementation

* Mask factors into classifiers
* Assume these factors are predictive over a month and predict returns 20 days into the future
* Re-weight individual factors in aggregate based on performance
* Profit?

## Dynamically-Weighted Signal with AdaBoost

* Build signal in research
    * scikit-learn `AdaBoostClassifier`
* 50/50 train/test split

## Testing the Signal

<img src="./images/train.png" alt="AdaBoost Accuracy" style="width: 800px;"/>

<img src="./images/test.png" alt="AdaBoost Accuracy" style="width: 800px;"/>

## Feature Importances

<img src="./images/feature_importances.png" alt="AdaBoost Features" style="width: 800px;"/>

## Dynamically-Weighted Signal with AdaBoost

<img src="./images/boosted_factor_portfolio.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Dynamically-Weighted Signal with AdaBoost

<img src="./images/boosted_factor_portfolio_curve.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Dynamically-Weighted Signal with AdaBoost

<img src="./images/boosted_factor_portfolio_fingers.png" alt="Equal-weighted Portfolio" style="width: 800px;"/>

## Market Impact

* It is not enough to find a signal
    * Have to check if it is viable
* Take into account commissions costs
* Take into account slippage costs

<img src="./images/boosted_backtest_curve.png" alt="Boosted factor in the backtester" style="width: 800px;"/>

## A Potential Culprit

<img src="./images/boosted_backtest_turnover.png" alt="Boosted factor in the backtester" style="width: 800px;"/>

## Limitations of Model

* Garbage in, garbage out
    * Passing in better factors should ideally improve performance
* AdaBoost is sensitive to noisy data and outliers
    * Financial data is ugly
    * A different ensemble classifier might improve performance
* Maybe just more weak classifiers

## Limitations of Platform

* Limited training window for ML classifier
* Limited backtest window

## Tools

* Alphalens: https://github.com/quantopian/alphalens
* Pyfolio: https://github.com/quantopian/pyfolio
* Zipline: https://github.com/quantopian/zipline
* scikit-learn: https://github.com/scikit-learn/scikit-learn
* Machine Learning on Quantopian forum post series:
    * https://www.quantopian.com/posts/machine-learning-on-quantopian
    * https://www.quantopian.com/posts/machine-learning-on-quantopian-part-2-ml-as-a-factor
    * https://www.quantopian.com/posts/machine-learning-on-quantopian-part-3-building-an-algorithm


* Quantopian Research: https://www.quantopian.com/research
* Quantopian Lecture Series: https://www.quantopian.com/lectures

<div style="text-align:center">
    <center><img src="./images/twitter_logo.png" alt='Twitter' style="width: 50px;"/></center>
    <h3>@clean_utensils</h3>
</div>

<div style="text-align:center">
    <center><img src="./images/github_logo.png" alt="Github" style="width: 50px;"/></center>
    <h3>@mmargenot</h3>
</div>

<div style="text-align:center">
    <center><img src="./images/logo-q-press-logo-red.png" alt="Quantopian" style="width: 100px;"/></center>
    <h3>max@quantopian.com</h3>
</div>

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company.  In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*

In [1]:
!jupyter nbconvert --to slides ./Thrifting_Alpha_Talk.ipynb --reveal-prefix=http://cdnjs.cloudflare.com/ajax/libs/reveal.js/3.2.0

[NbConvertApp] Converting notebook ./Thrifting_Alpha_Talk.ipynb to slides
[NbConvertApp] Writing 280921 bytes to ./Thrifting_Alpha_Talk.slides.html
